In [60]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta

In [61]:
load_dotenv()

True

In [62]:
# Set adjustable variables
seven_day_delta = timedelta(days=7)
one_day_delta = timedelta(days=1)

#Set the end date as yesterday
end_date_time = datetime.today() - one_day_delta
end_date = end_date_time.date()

#Set the start date as end date - 7 days
start_date_time = end_date_time - seven_day_delta
start_date = start_date_time.date()


##Set the time to 1PM for tweet retrieval

start_date_time = start_date_time.replace(minute=0, hour=13, second=0)
end_date_time = end_date_time.replace(minute=0, hour=13, second=0)
end_date_time

datetime.datetime(2022, 5, 14, 13, 0, 0, 620018)

In [63]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
twitter_bearer_token = os.getenv("TWITTER_BEARER_TOKEN")


In [64]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [65]:
# Format current date as ISO format
start_date = pd.Timestamp(start_date, tz="America/New_York").isoformat()
end_date = pd.Timestamp(end_date, tz="America/New_York").isoformat()

timeframe = "1Day"

In [66]:
ticker = input("Enter the ticker for the analysis : ")

Enter the ticker for the analysis :  TSLA


In [67]:
ticker

'TSLA'

In [68]:
df_stock_all = alpaca.get_bars(
    ticker,
    timeframe,
    start = start_date,
    end = end_date
).df
df_stock_all

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2022-05-09 04:00:00+00:00,836.41,845.6300,781.150,787.11,30196155,1129573,810.107982
2022-05-10 04:00:00+00:00,818.87,825.3600,774.250,800.04,28029762,890613,800.397258
2022-05-11 04:00:00+00:00,796.01,809.7700,727.200,734.00,32251681,1137179,765.927738
2022-05-12 04:00:00+00:00,701.10,759.6600,680.000,728.00,46672183,1389935,720.962135
2022-05-13 04:00:00+00:00,773.99,787.3499,751.565,769.59,30667186,862794,769.716998


In [69]:
# Set the Bearer Token for Authorisation for Twitter API call.
auth_token = "Bearer " + twitter_bearer_token
headers = {"Authorization": auth_token}

## Function to read 100 recent tweets related to ticker and from the date time specified.
def read_100_Tweets(ticker, tweet_date_time):
    ## Function to read 100 recent tweets from the specific dates
    ##  Input : ticker - Stock Ticker
    ##          tweet_date_time - UTC Date/Time Format YYYY-MM-DDTHH:mm:ssZ (ISO 8601/RFC 3339)
    ## 
    ##  Output: List of 100 tweets
    twitter_api_url  = f"https://api.twitter.com/2/tweets/search/recent?max_results=100&query={ticker}&start_time={tweet_date_time}"
    response = requests.get(twitter_api_url, headers=headers)
    ## Check for 200 status code which means it was successful
    tweets_list = [];
    if(response.status_code == 200):
        json_response = response.json()
        #Check if there are any tweets at all
        if('data' in json_response.keys()): 
            all_tweets = response.json()["data"]
            for tweet in all_tweets:
                tweets_list.append(tweet["text"])
            
    else:
        print(f"Response code: {response.status_code}.  Error in getting the tweet");
    return tweets_list    

In [70]:
## Perform Vader Sentiment Analysis

## Define Sentiment Object for Sentiment Analysis

sentiment_obj = SentimentIntensityAnalyzer()
def perform_sentiment_analysis(tweets_list):
    ## Function to read 100 recent tweets from the specific dates
    ##  Input : tweets_list - List of 100 tweets
    ##          
    ## 
    ##  Output: sentiment score average
    
    ##Check if there are tweets to analyse
    if (len(tweets_list) > 0):
        sentiment_scores_all = []
        for tweet in tweets_list:
            sentiment_dict = sentiment_obj.polarity_scores(tweet)
            sentiment_scores_all.append(sentiment_dict["compound"])

        #Average the sentiment of all tweets    
        average_sentiment = np.average(sentiment_scores_all)
        return average_sentiment
    else:
        return 0;
    

In [71]:
tweet_sentiments = []
analysis_date_time = start_date_time
while analysis_date_time <= end_date_time:
    delta = timedelta(days=1)
    print(f"Executing Tweet Analysis for {ticker} on {analysis_date_time.isoformat()}")
    tweets_list = read_100_Tweets(ticker, analysis_date_time.isoformat() + "Z")
    sentiment_score = perform_sentiment_analysis(tweets_list)
    tweet_sentiment = {}
    tweet_sentiment["Ticker"] = ticker
    tweet_sentiment["Date"] = analysis_date_time
    tweet_sentiment["Sentiment_Score"] = sentiment_score
    tweet_sentiments.append(tweet_sentiment)
        #print("Ticker: " + tweet_sentiment["ticker"] + ", Date : " + str(tweet_sentiment["date"]) + ", Sentiment Score: " + str(tweet_sentiment["sentiment_score"]))
    analysis_date_time += delta

Executing Tweet Analysis for TSLA on 2022-05-07T13:00:00.620018
Response code: 400.  Error in getting the tweet
Executing Tweet Analysis for TSLA on 2022-05-08T13:00:00.620018
Executing Tweet Analysis for TSLA on 2022-05-09T13:00:00.620018
Executing Tweet Analysis for TSLA on 2022-05-10T13:00:00.620018
Executing Tweet Analysis for TSLA on 2022-05-11T13:00:00.620018
Executing Tweet Analysis for TSLA on 2022-05-12T13:00:00.620018
Executing Tweet Analysis for TSLA on 2022-05-13T13:00:00.620018
Executing Tweet Analysis for TSLA on 2022-05-14T13:00:00.620018


In [72]:
tweet_sentiments

[{'Ticker': 'TSLA',
  'Date': datetime.datetime(2022, 5, 7, 13, 0, 0, 620018),
  'Sentiment_Score': 0},
 {'Ticker': 'TSLA',
  'Date': datetime.datetime(2022, 5, 8, 13, 0, 0, 620018),
  'Sentiment_Score': 0.12281099999999999},
 {'Ticker': 'TSLA',
  'Date': datetime.datetime(2022, 5, 9, 13, 0, 0, 620018),
  'Sentiment_Score': 0.12281099999999999},
 {'Ticker': 'TSLA',
  'Date': datetime.datetime(2022, 5, 10, 13, 0, 0, 620018),
  'Sentiment_Score': 0.12281099999999999},
 {'Ticker': 'TSLA',
  'Date': datetime.datetime(2022, 5, 11, 13, 0, 0, 620018),
  'Sentiment_Score': 0.12281099999999999},
 {'Ticker': 'TSLA',
  'Date': datetime.datetime(2022, 5, 12, 13, 0, 0, 620018),
  'Sentiment_Score': 0.12281099999999999},
 {'Ticker': 'TSLA',
  'Date': datetime.datetime(2022, 5, 13, 13, 0, 0, 620018),
  'Sentiment_Score': 0.12281099999999999},
 {'Ticker': 'TSLA',
  'Date': datetime.datetime(2022, 5, 14, 13, 0, 0, 620018),
  'Sentiment_Score': 0.12281099999999999}]

In [73]:
stock_tweet_sentiment_df = pd.DataFrame(tweet_sentiments)
#Date Clean up
stock_tweet_sentiment_df["Date"] = pd.to_datetime(stock_tweet_sentiment_df["Date"].dt.date)
#Check Date Type

In [73]:
stock_tweet_sentiment_df.dtypes

In [74]:
stock_tweet_sentiment_df

,Ticker,Date,Sentiment_Score
0,TSLA,2022-05-07,0.000000
1,TSLA,2022-05-08,0.122811
2,TSLA,2022-05-09,0.122811
3,TSLA,2022-05-10,0.122811
4,TSLA,2022-05-11,0.122811
5,TSLA,2022-05-12,0.122811
6,TSLA,2022-05-13,0.122811
7,TSLA,2022-05-14,0.122811
